CAPTURA INDIVIDUAL DE IMÁGENES DESDE CÁMARAS PARA CALIBRACIÓN

Funcionalidad principal:
- Conexión a una cámara IP a través de una URL tipo MJPEG (ej. desde DroidCam)
- Visualización en vivo de la transmisión de video en una ventana OpenCV
- Captura manual de imágenes al presionar la tecla "c"
- Almacenamiento automático de imágenes numeradas secuencialmente en una carpeta específica
- Finalización del proceso mediante la tecla "q" o "ESC"

Implementación técnica:
- Camera (clase principal):
    - init():
        - Establece la conexión a la cámara IP usando su dirección y un identificador (nombre)
        - Verifica que la conexión se haya abierto correctamente
        - Si falla la conexión, el programa se detiene
    - getNextIndex():
        - Escanea la carpeta de imágenes para encontrar el número de índice más alto ya existente
        - Usa una expresión regular para detectar nombres tipo 000001.bmp
        - Devuelve el siguiente número disponible para guardar la imagen sin sobrescribir
    - saveSequence():
        - Crea el directorio si no existe
        - Inicia una ventana de visualización en tiempo real del video
        - Permite al usuario guardar imágenes con tecla 'c'
        - Las imágenes se guardan como archivos secuenciales (i.e. 000001.bmp.)
        - Permite salir del bucle con "q" o "ESC"
        - Al finalizar, libera los recursos de la cámara y cierra la ventana

Modo de ejecución:
- Se definen las IPs de las cámaras IP (izquierda y derecha)
- Se crea una instancia Camera para cada una
- Se llama al método saveSequence() indicando el directorio y extensión

Autores:
- Belda Martínez, Marcos
- Espert Cornejo, Ángela
- Francés Llimerá, Lourdes

In [1]:
import os   # For handling directories and files
import cv2  # OpenCV: for video capture and image handling
import re   # For regular expressions, used to match image file names

In [2]:
class Camera:
    """
    Class to manage an IP camera and manually capture individual images.
    """
    
    def __init__(self, name, ip):
        """
        Initializes connection to the IP camera.
        """
        self.name = 'Camera_' + name                       # Internal identifier
        self.url = f"http://{ip}:8080"                     # Stream URL
        self.capture = cv2.VideoCapture(self.url)          # Start capture
        
        # Check if connection was successful
        if not self.capture.isOpened():
            print(f"ERROR: Could not connect to camera at {self.url}")
            exit()
            
    def getNextIndex(self, dir):
        """
        Determines the next available image index in the given directory.

        Looks for files matching the pattern ######.bmp/jpg/jpeg and returns
        the next available number.
        """
        namePattern = re.compile(r'^(\d{6})\.(bmp|jpg|jpeg)$', re.IGNORECASE)
        max_idx = -1 
        
        for filename in os.listdir(dir):
            match = namePattern.match(filename) # Search pattern coincidence
            if match:
                idx = int(match.group(1))       # Gets image number
                if idx > max_idx:
                    max_idx = idx               # Updates max index
        return max_idx + 1                      # Returns next available number
    
    def saveSequence(self, sequence, ext):
        """
        Displays the video stream and lets the user save frames manually.
        """
        # Create directory if it doesn't exist
        if not os.path.exists(sequence):
            os.makedirs(sequence)
            
        # Determinate number of the image to save
        idx = self.getNextIndex(sequence)
        
        window =  f"Camera - {self.name}"   # Window name
        cv2.namedWindow(window)             # Create window
        
        print("Press 'c' to capture image, 'q' or ESC to quit.")
        
        while True:
            ret, frame = self.capture.read()        # Blabla
            if not ret:
                print("ERROR: Could not capture frame.")
                break                               # Exit the loop
            
            cv2.imshow(window, frame)               # Show the video
            
            key = cv2.waitKey(1) & 0xFF             # Wait for a key
            if key == ord('c'):
                filename = os.path.join(sequence, f"{idx:06d}.{ext}")
                # Try saving the image
                if cv2.imwrite(filename, frame):
                    print(f"Image saved: {filename}")
                    idx += 1
                else:
                    print("ERROR: Failed to save image.")
                    
            elif key == 27 or key == 113:
                print("Exiting and releasing resources...")
                break
            
        # Free resources and close windows
        self.capture.release()
        cv2.destroyAllWindows()
        print("Capture session finished.")
                    
            

In [ ]:
# IPs of the cameras to use
ip_R = "192.168.1.132"  # Right camera IP 
extension = "bmp"  # Image file extension

# Create camera instances
camera_R = Camera("DroidCam_R", ip_R)

# Start individual capture sessions
# It is recommended to do this separately to avoid overlapping windows
camera_R.saveSequence("right_calibration_images", extension)

In [3]:
# IPs of the cameras to use
ip_L = "192.168.1.129"  # Left camera IP
extension = "bmp"  # Image file extension

# Create camera instances
camera_L = Camera("DroidCam_L", ip_L)

# Start individual capture sessions
# It is recommended to do this separately to avoid overlapping windows
camera_L.saveSequence("left_calibration_images", extension)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/marcos/non-snap/miniconda3/envs/v3d_2425_env/lib/python3.11/site-packages/cv2/qt/plugins"


Press 'c' to capture image, 'q' or ESC to quit.
Image saved: left_calibration_images/000000.bmp
Image saved: left_calibration_images/000001.bmp
Image saved: left_calibration_images/000002.bmp
Image saved: left_calibration_images/000003.bmp
Image saved: left_calibration_images/000004.bmp
Image saved: left_calibration_images/000005.bmp
Image saved: left_calibration_images/000006.bmp
Image saved: left_calibration_images/000007.bmp
Image saved: left_calibration_images/000008.bmp
Image saved: left_calibration_images/000009.bmp
Image saved: left_calibration_images/000010.bmp
Image saved: left_calibration_images/000011.bmp
Image saved: left_calibration_images/000012.bmp
Image saved: left_calibration_images/000013.bmp
Image saved: left_calibration_images/000014.bmp
Image saved: left_calibration_images/000015.bmp
Image saved: left_calibration_images/000016.bmp
Exiting and releasing resources...
Capture session finished.
